### Deploy Master Firebase

In [1]:
#!pip install google-cloud-storage

In [2]:
#!pip install firebase_admin

In [3]:
#!pip install firebase

In [4]:
#!pip install --upgrade pip

In [5]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [6]:
#!pip install pyrebase4

In [7]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [8]:
#import pyrebase
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import shutil

In [9]:
#import firebase_admin
#from firebase_admin import credentials

cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
app = firebase_admin.initialize_app(cred)

In [10]:
fire_db = firestore.client()

#### Read Data

In [1]:
data_master = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Deploy Result/Master 17052022.xlsx")
data_master

<IPython.core.display.Javascript object>

,surah,ayah,word rank,juz,page,surah arabic,surah latin,word count ayah,letter count ayah,tanzil_clean,...,twogram_bangla,twogram_russian,threegram_string,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
0,1,1,1,1,1,الفاتحة,Al-Fatiha,4,19,بسم,...,আল্লাহর নামে,Во имя Бога,بسم الله الرحمن,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
1,1,1,2,1,1,الفاتحة,Al-Fatiha,4,19,الله,...,পরম করুণাময় ঈশ্বর,Бог Милостивый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,1,1,الفاتحة,Al-Fatiha,4,19,الرحمن,...,পরম করুণাময়,Самый Милосердный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,4,1,1,الفاتحة,Al-Fatiha,4,19,الرحيم,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,1,1,1,الفاتحة,Al-Fatiha,4,18,الحمد,...,সৃষ্টিকর্তাকে ধন্যবাদ,Слава Богу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78242,114,5,4,30,604,الناس,An-Nas,5,20,صدور,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78243,114,5,5,30,604,الناس,An-Nas,5,20,الناس,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78244,114,6,1,30,604,الناس,An-Nas,3,13,من,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78245,114,6,2,30,604,الناس,An-Nas,3,13,الجنة,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Sending Data To Firestore

##### Page Data

In [21]:
#data_root = data_root.iloc[0:3150,]  # data divide because of quota
data_root = data_root.drop_duplicates(subset = ['arabic_simple', 'arabic'], keep = 'last').reset_index(drop = True)
data_root

,sequence,root,root_arabic,root_arabic_frequency,surah,ayah,word rank,surah latin,arabic_simple,arabic,arabic_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,qwl,قول,1722,2,26,23,Al-Baqara,فيقولون,فَيَقُولُونَ,1,[thus] they will say,تو وہ کہتے ہیں,तो वो कहते हैं,maka mereka berkata,তারা বলে তখন,derler ki,то они говорят
1,1,qwl,قول,1722,2,102,29,Al-Baqara,يقولا,يَقُولَا,1,they [both] say,وہ دونوں کہتے,वो दोनों कहते,berkata keduanya,দুজনে বলেছে,demedikçe,говорили они двое:
2,1,qwl,قول,1722,2,118,17,Al-Baqara,قولهم,قَوْلِهِمْ,2,their saying,ان کی بات کی,उनकी बात के,ucapan mereka,তাদের কথার,onların sözlerinin,их словам
3,1,qwl,قول,1722,2,263,1,Al-Baqara,قول,قَوْلٌ,1,A word,بات,बात,perkataan,কথা,bir söz (söylemek),Речь
4,1,qwl,قول,1722,3,147,3,Al Imran,قولهم,قَوْلَهُمْ,1,their words,ان کی بات,बात उनकी,perkataan mereka,তাদের কথা,sözleri,"речью их,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6204,99,nhr,نهر,113,73,20,19,Al-Muzzammil,والنهار,وَالنَّهَارَ,12,and the day,اور دن کو,और दिन को,dan siang,ও দিনকে,ve gündüzü,"и день,"
6205,99,nhr,نهر,113,78,11,2,An-Naba',النهار,النَّهَارَ,12,the day,دن کو,दिन को,siang,দিনকে,gündüzü,день
6206,99,nhr,نهر,113,92,2,1,Al-Lail,والنهار,وَالنَّهَارِ,13,and the day,اور دن کو,और दिन के,dan siang,ও দিনের,ve gündüzün,"и дня,"
6207,99,nhr,نهر,113,93,10,4,Ad-Dhuha,تنهر,تَنْهَرْ,1,repel,جھڑکو,आप झिड़कये (उसे),kamu menghardik,তিরস্কার করো,azarlama,отгоняй!


In [22]:
# root
sequence_list = data_root["sequence"].to_list()
root_arabic_list = data_root["root_arabic"].to_list()
root_list = data_root["root"].to_list()
root_arabic_freq_list = data_root["root_arabic_frequency"].to_list()
# word 
arabic_simple_list = data_root["arabic_simple"].to_list()
arabic_list = data_root["arabic"].to_list()
trans_eng_list = data_root["translate_english"].to_list()
trans_urdu_list = data_root["translate_urdu"].to_list()
trans_hindi_list = data_root["translate_hindi"].to_list()
trans_indo_list = data_root["translate_indonesian"].to_list()
trans_bangla_list = data_root["translate_bangla"].to_list()
trans_tur_list = data_root["translate_turkish"].to_list()
trans_rus_list = data_root["translate_russian"].to_list()
#surah_list = data_root["surah"].to_list()
#ayah_list = data_root["ayah"].to_list()
#word_rank_list = data_root["word rank"].to_list()

In [23]:
#firestore_sending_root_data = zip(sequence_list,root_arabic_list,root_list,root_arabic_freq_list,arabic_simple_list,arabic_list,surah_list,ayah_list,word_rank_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)
firestore_sending_root_data = zip(sequence_list,root_arabic_list,root_list,root_arabic_freq_list,arabic_simple_list,arabic_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [24]:
# Sending Root Data With Array
root_check = 1
data_list = []
data = dict()
data_all_list = []
#for sequence,root_arabic,root,root_arabic_freq,arabic_simple,arabic,surah,ayah,word_rank,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_root_data:
for sequence,root_arabic,root,root_arabic_freq,arabic_simple,arabic,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_root_data:
    if sequence == root_check:
        data_var = {"root_arabic":root_arabic,
            "root":root,
            "root_arabic_freq":root_arabic_freq,
            "arabic_simple":arabic_simple,
            "arabic":arabic,
            #"surah":surah,
            #"ayah":ayah,
            #"word_rank":word_rank,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus
            }
    
        data_list.append(data_var)
        #data["root"] = data_list
    else:
        data_all_list.append(data_list)
        #fire_db.collection("Root_Data").document(f"root{sequence}").set(data)    
        root_check += 1
        data_list = []
        data_var = {"root_arabic":root_arabic,
            "root":root,
            "root_arabic_freq":root_arabic_freq,
            "arabic_simple":arabic_simple,
            "arabic":arabic,
            #"surah":surah,
            #"ayah":ayah,
            #"word_rank":word_rank,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus
            }

        data_list.append(data_var)
    
    #fire_db.collection("Root_Data").document(f"root{sequence}").collection(f"{root}").document(f"{arabic_simple}").set(data)
    #fire_db.collection("Root_Data").document(f"root{sequence}").set(data)
    
    # Not: Eger document ile bitiyorsa data set ile gönderiliyor.
    #fire_db.collection("Root_Data").document(f"root{sequence}").collection("Detail").add(data)  
     
    # Not: Eğer collection ile bitiyorsa data add ile gönderiliyor ve collection dan sonraki document adları rastgele oluşturuluyor.                                                                                             

In [25]:
for i in range(len(data_all_list)):
    data["root"] = data_all_list[i]
    fire_db.collection("root_data").document(f"root{i+1}").set(data)

In [22]:
#fire_db.collection(u'cities').document(u'DC').delete()